<a href="https://colab.research.google.com/github/luizaugustoliveira/Sistema-Feedbacks/blob/main/AgentesInteligentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

instalação do orquestrador de agentes que é o langchain

In [ ]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00


configuração das chaves que dão acesso aos modelos da IA do Google

In [ ]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

conexão com o google gemini

In [ ]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature=1,
    api_key=GOOGLE_API_KEY
)

In [ ]:
resposta_teste  = llm.invoke("Quem é você? Seja criativo")
print(resposta_teste.content)

In [ ]:
FEEDBACK_ROUTER_PROMPT = (
    "Você é um Analista de Produto, especialista em interpretar a voz do cliente."
    "Seu objetivo é analisar o feedback de um usuário e direcioná-lo para a equipe correta para ação."
    "Analise o feedback e retorne SOMENTE um JSON com sua análise:\n"
    "{\n"
    '  "pensamento": "...",\n'
    '  "acao_sugerida": "ENVIAR_PARA_PRODUTO" | "ENVIAR_PARA_SUPORTE" | "ENVIAR_PARA_MARKETING",\n'
    '  "nivel_prioridade": "ALTA" | "MEDIA" | "BAIXA",\n'
    '  "dados_adicionais": {"tipo_feedback": "Sugestão de Feature | Relato de Bug | Elogio | Reclamação"}\n'
    "}\n\n"
    "## Diretrizes de Decisão:\n"
    "1.  **ENVIAR_PARA_PRODUTO**: O feedback contém uma sugestão de nova funcionalidade ('seria legal se...'), ou uma crítica construtiva sobre o design/usabilidade.\n"
    "2.  **ENVIAR_PARA_SUPORTE**: O usuário está relatando um erro, um bug, ou algo que não está funcionando como esperado ('não consigo clicar no botão...'). Prioridade ALTA se impedir o uso.\n"
    "3.  **ENVIAR_PARA_MARKETING**: O usuário está fazendo um elogio sobre o produto, o que pode ser usado como um depoimento, ou está comparando o produto com concorrentes.\n"
)

Bibliotecas que ajudam o output a ser mais estruturado

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class FeedbackOut(BaseModel):
  acao_sugerida: Literal["ENVIAR_PARA_PRODUTO" , "ENVIAR_PARA_SUPORTE" , "ENVIAR_PARA_MARKETING"]
  nivel_prioridade: Literal["ALTA" , "MEDIA" , "BAIXA"]
  dados_adicionais: List[str] = Field(default_factory=list)

In [ ]:
llm_feedback = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature=0,
    api_key=GOOGLE_API_KEY
)

Bibliotecas do langchain pro modelo executar aquilo que a gente quer com a saída estruturada definida anteriormente

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

feedback_chain = llm_feedback.with_structured_output(FeedbackOut)

def feedback(mensagem: str) -> Dict:
  saida:FeedbackOut = feedback_chain.invoke([
      SystemMessage(content=FEEDBACK_ROUTER_PROMPT),
      HumanMessage(content=mensagem)
  ])
  return saida.model_dump()


In [ ]:
testes_feedback = [
    # Categoria 1: Casos Diretos e Claros
    "Seria incrível se o app tivesse um modo escuro. Meus olhos agradeceriam!",
    "Não consigo fazer o login. Clico no botão 'Entrar' e nada acontece.",
    "Eu amo a nova atualização! O design ficou muito mais limpo e rápido. Parabéns para a equipe!",

    # Categoria 2: Casos Mistos e Ambíguos
    "O novo menu é muito confuso. Levei cinco minutos para encontrar a página de configurações. O antigo era melhor.",
    "A funcionalidade de exportar relatórios é a melhor que já vi! Mas seria ainda mais perfeita se permitisse exportar para PDF também.",
    "ESTOU FURIOSO! O app fechou sozinho no meio de um trabalho importante e perdi tudo. Isso é inaceitável!",

    # Categoria 3: Casos de Borda e Fora de Escopo
    "Não gostei.",
    "Gostaria de saber por que minha fatura veio com um valor diferente este mês.",
    "O app de vocês é bom, mas o Concorrente X tem uma função de colaboração em tempo real que é muito superior.",
    "Qual é a previsão do tempo para amanhã?",

    "O sistema não está aberto para a matrícula, qual o período de matrícula?"
]


In [ ]:
for msg_teste in testes_feedback:
  print(f"Mensagem: {msg_teste}\n -> Resposta: {feedback(msg_teste)}\n")

Mensagem: Seria incrível se o app tivesse um modo escuro. Meus olhos agradeceriam!
 -> Resposta: {'acao_sugerida': 'ENVIAR_PARA_PRODUTO', 'nivel_prioridade': 'MEDIA', 'dados_adicionais': ['tipo_feedback: Sugestão de Feature']}

Mensagem: Não consigo fazer o login. Clico no botão 'Entrar' e nada acontece.
 -> Resposta: {'acao_sugerida': 'ENVIAR_PARA_SUPORTE', 'nivel_prioridade': 'ALTA', 'dados_adicionais': ['tipo_feedback: Relato de Bug']}

Mensagem: Eu amo a nova atualização! O design ficou muito mais limpo e rápido. Parabéns para a equipe!
 -> Resposta: {'acao_sugerida': 'ENVIAR_PARA_MARKETING', 'nivel_prioridade': 'BAIXA', 'dados_adicionais': ['{"tipo_feedback": "Elogio"}']}

Mensagem: O novo menu é muito confuso. Levei cinco minutos para encontrar a página de configurações. O antigo era melhor.
 -> Resposta: {'acao_sugerida': 'ENVIAR_PARA_PRODUTO', 'nivel_prioridade': 'MEDIA', 'dados_adicionais': ['tipo_feedback: Reclamação']}

Mensagem: A funcionalidade de exportar relatórios é a m